## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model =ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002264A717E90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002264A8855D0>, model_name='gemma2-9b-it')

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hi, My name is Aswani and i can be a nice Ai Engineer.")])

AIMessage(content="Hi Aswani, it's nice to meet you!  \n\nThat's great to hear you're interested in becoming an AI Engineer. It's a fascinating and growing field with lots of opportunities to make a real impact.  \n\nWhat are some of the things that interest you most about AI engineering?  Perhaps we could chat about some of the challenges and rewards of the profession.\n", response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 25, 'total_tokens': 108, 'completion_time': 0.150909091, 'prompt_time': 0.00140043, 'queue_time': 0.258884219, 'total_time': 0.152309521}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-09c0b025-7417-4630-9806-7139e3a9ba5a-0', usage_metadata={'input_tokens': 25, 'output_tokens': 83, 'total_tokens': 108})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hi, My name is Aswani and i a good ai Engineer."),
        AIMessage(content = "Hi Aswani, it's nice to meet you!  \n\nIt's great to hear that you're interested in becoming an AI Engineer. It is a fascinating and rapidly growing field with lots of opportunities to make a real impact. What aspects of AI engineering are you most interested in? Do you have any experience or projects you've worked on already? I'd love to hear more about your journey!\n"),
        HumanMessage(content="Hey What's my name and what do i do ?")
    ]
)

AIMessage(content="You told me your name is Aswani and that you are a good AI Engineer!  😊  \n\nIs there anything else you'd like to tell me about yourself or discuss regarding AI?  I'm here to chat. \n", response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 131, 'total_tokens': 182, 'completion_time': 0.092727273, 'prompt_time': 0.003363412, 'queue_time': 0.257335818, 'total_time': 0.096090685}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2071d432-b753-4614-b788-9431b5314c86-0', usage_metadata={'input_tokens': 131, 'output_tokens': 51, 'total_tokens': 182})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Defining a function to make sure that one session is completely different
# from other session. its return type is basechatmessagehistory
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config = {"configurable":{"session_id": "chat1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Aswani and i am a good ai Engineer.")],
    config=config
)

In [8]:
response.content

"Hi Aswani, it's nice to meet you! That's great to hear that you're a good AI Engineer. \n\nWhat kind of AI work do you do?  I'm always interested in learning more about what people are building with AI.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name ?")],
    config=config
)

AIMessage(content='Your name is Aswani.  \n\nI remember that from our introduction! 😊  Is there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 96, 'total_tokens': 126, 'completion_time': 0.054545455, 'prompt_time': 0.002725771, 'queue_time': 0.255119459, 'total_time': 0.057271226}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e7203e91-305e-478f-bdf0-cc6f8b1f979e-0', usage_metadata={'input_tokens': 96, 'output_tokens': 30, 'total_tokens': 126})

In [10]:
## change the config -->session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name ?")],
    config=config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [11]:
# Giving data to config1
with_message_history.invoke(
    [HumanMessage(content="My name is Kaite")],
    config=config1
)

AIMessage(content="Hi Kaite! It's nice to meet you.  What can I do for you today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 68, 'total_tokens': 94, 'completion_time': 0.047272727, 'prompt_time': 0.002280881, 'queue_time': 0.252624369, 'total_time': 0.049553608}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-89518427-7999-4652-8d12-42c61bf7b120-0', usage_metadata={'input_tokens': 68, 'output_tokens': 26, 'total_tokens': 94})

In [12]:
## change the config -->session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name ?")],
    config=config1
)

response.content

'Your name is Kaite!  I remember now. 😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a best mental health Assistant. Help people to get rid of their mental stress and anxiety."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Aswani.")]})

AIMessage(content="Hi Aswani, it's great to meet you. I'm here to help you manage your mental stress and anxiety. \n\nRemember, I'm not a human therapist and can't provide medical advice. But, I can offer support, information, and techniques that people find helpful.\n\nTo start, tell me a little about what's been causing you stress and anxiety lately. \n\nThe more specific you are, the better I can understand your situation and offer relevant support. For example, you could tell me about:\n\n* **Specific events or situations** that are causing you worry\n* **Physical symptoms** you're experiencing (e.g., racing heart, difficulty sleeping)\n* **Thought patterns** you've noticed (e.g., constantly worrying, negative self-talk)\n* **Coping mechanisms** you've tried so far \n\nI'm here to listen without judgment and help you explore ways to feel better.  We can work together on this! 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 213, 'prompt_tokens': 36, 'total_tokens':

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is aswani")],
    config=config
)

response

AIMessage(content="Hi Aswani, it's nice to meet you. I understand you're looking for help with mental stress and anxiety. While I'm not a therapist or medical professional, I can offer some strategies and support that might be helpful.  \n\nRemember, I'm here to listen and offer guidance, but if your struggles are severe or persistent, please reach out to a qualified mental health professional. \n\nTo get started, tell me a little more about what you're experiencing. \n\n* **What are some of the specific things that are causing you stress and anxiety?**\n* **How long have you been feeling this way?**\n* **Have you tried any coping mechanisms already?**\n\nThe more information you share, the better I can understand your situation and offer relevant advice.  \n\nHere are some general tips that may be helpful in the meantime:\n\n* **Practice mindfulness:** Pay attention to the present moment without judgment. Try deep breathing exercises or meditation.\n* **Engage in physical activity:** 

In [17]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a best mental health Assistant. Help people to get rid of their mental stress and anxiety in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [18]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is aswani")],"language":"Hindi"})
response.content

'नमस्ते आश्वनी! \n\nयह सुनकर अच्छा लगा कि आप मुझसे मदद लेने के लिए आए हैं। तनाव और चिंता से जूझना आसान नहीं होता, लेकिन आप अकेले नहीं हैं। बहुत से लोग इन समस्याओं का सामना करते हैं, और मदद लेना एक बड़ी ताकत है। \n\nमुझे बताओ, आश्वनी, आप किस तरह का तनाव और चिंता का अनुभव कर रहे हैं? क्या कोई खास घटना हुई है, या यह लगातार चल रहा है? \n\nमुझे आपके अनुभवों को समझने में मदद करने के लिए कुछ सवाल पूछना चाहूंगा। \n\nअपनी बात साझा करने से आपको राहत मिल सकती है, और हम मिलकर कुछ उपाय खोज सकते हैं। \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [20]:
config2 = {"configurable": {"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi, i am aswani")],"language":"Hindi"},
    config=config2
)
response.content

'Namaste Aswani, 😊 \n\nMujhe khush hota hai ki aap yahan hain!  Main aapka mental health assistant hoon aur aapke stress aur anxiety ko kam karne mein aapka saath dene ke liye taiyar hoon.\n\nKya aap mujhe bata sakte hain ki aap kya jaanana chahte hain?  \n\nYa phir aap kya khayal kar rahe hain jo aapko stress aur anxiety dikha raha hai? \n\nYaad rakhein, aap akele nahi hain.  Bahut saare log aise hi hisaab rakhte hain.  Hamari baat karenge aur aapko achha lagne mein madad karne ki koshish karungi. 💪 \n\n'

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name is ?")],"language":"german"},
    config=config2
)
response.content

'Your name is Aswani. 😊 \n\nWie kann ich dir heute weiterhelfen, Aswani? \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [22]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = True,
    start_on = "human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\aswan\anaconda3\envs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [23]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What ice cream do i like ?")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your favorite ice cream flavor.  \n\nWhat's your favorite flavor? 😊  I love learning new things!\n"

In [24]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What math problem did i asked to you ?")],
        "language": "English"
    }
)
response.content

'You asked me "What is 2 + 2?" 😊 \n\n\nLet me know if you want to try another one!\n'

In [25]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config5={"configurable": {"session_id":"chat5"}}


In [26]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name is ?")],
     "language":"English"},
    config=config5
)
response.content

"It's lovely to meet you!  \n\nWhile I can offer helpful tips and strategies, remember that I'm not a therapist or doctor. If you're struggling with severe anxiety or stress, please reach out to a mental health professional. \n\nNow, tell me, what's on your mind? What kind of stress or anxiety are you experiencing? \n\nKnowing more about your situation will help me offer more personalized advice.  \n\nAnd to answer your question, I don't know your name. Would you like to share it with me? 😊  \n\n"